# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [45]:
# Dependencies and Setup

import pandas as pd
import requests
import json



In [46]:

# zones for electrical utilities in Southwest
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO"]


In [47]:

# get carbon intensity history for the SW utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [48]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [49]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
sw_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_NM = pd.DataFrame.from_dict(sw_pnm1,orient='index')


In [50]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_NM[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'uknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_NM_new = df_NM.transpose()

# check data types
df_NM_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_26316\2417422126.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_NM[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,384,384,384,384,384,384,384,384,384,384,384,0,384,384,384,384,384
unique,15,24,110,22,25,230,265,193,169,308,5,0,2,17,62,341,2
top,US-SW-AZPS,2024-07-12T19:00:00.000Z,0,0,0,0,0,0,0,0,0,NaN,0,0,100,147,True
freq,48,16,216,360,336,96,50,146,78,24,374,NaN,381,367,24,3,207


In [51]:
# convert measured Energy values to integers in Giga Watts
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
                'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
                }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_NM_new = df_NM_new.astype(convert_dict)

#check that data types are changed to int
df_NM_new.dtypes

region                  object
datetime                object
nuclear                  int32
geothermal               int32
biomass                  int32
coal                     int32
wind                     int32
solar                    int32
hydro                    int32
gas                      int32
oil                      int32
unknown                 object
hydro-discharge          int32
battery_discharge        int32
renewable_percentage     int32
total_consumption        int32
estimated               object
dtype: object

In [52]:
# Date Time work

# import datetime dependencies
import dateutil.parser
from dateutil import tz
from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_NM_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_NM_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_NM_new['UTC time'] = times
df_NM_new['UTC date'] = dates
df_NM_new['UTC DateTime'] = DateTime

In [53]:
#set the UTC DateTime as the index
df_NM_new_reindex = df_NM_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_NM_newer = df_NM_new.drop('datetime', axis=1)
df_NM_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-07-12 19:00:00,US-SW-PNM,0,0,0,291,127,1146,17,648,0,NaN,0,0,58,2229,False,19:00:00,2024-07-12
2024-07-12 20:00:00,US-SW-PNM,0,0,0,317,140,1094,17,652,0,NaN,0,0,56,2220,False,20:00:00,2024-07-12
2024-07-12 21:00:00,US-SW-PNM,0,0,0,354,236,928,17,657,0,NaN,0,0,54,2192,False,21:00:00,2024-07-12
2024-07-12 22:00:00,US-SW-PNM,0,0,0,351,400,648,17,714,0,NaN,0,0,50,2132,False,22:00:00,2024-07-12
2024-07-12 23:00:00,US-SW-PNM,0,0,0,469,664,386,17,729,0,NaN,0,0,47,2267,False,23:00:00,2024-07-12


Section 3
Transform carbon intensity data

In [54]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
sw_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_NM_C = pd.DataFrame.from_dict(sw_pnm1C,orient='index')

In [55]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_NM_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_NM_C_new = df_NM_C.transpose()

# check data types
df_NM_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_26316\4108670115.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_NM_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [56]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_NM_C_new = df_NM_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_NM_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [57]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_NM_C_new['UTC time'] = times
df_NM_C_new['UTC date'] = dates
df_NM_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_NM_C_new_reindex = df_NM_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_NM_C_newer = df_NM_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [58]:
df_power_and_carbon= pd.merge(df_NM_newer, df_NM_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-07-12 19:00:00,US-SW-PNM,0,0,0,291,127,1146,17,648,0,NaN,0,0,58,2229,False,19:00:00,2024-07-12,333,False
2024-07-12 20:00:00,US-SW-PNM,0,0,0,317,140,1094,17,652,0,NaN,0,0,56,2220,False,20:00:00,2024-07-12,349,False
2024-07-12 21:00:00,US-SW-PNM,0,0,0,354,236,928,17,657,0,NaN,0,0,54,2192,False,21:00:00,2024-07-12,375,False
2024-07-12 22:00:00,US-SW-PNM,0,0,0,351,400,648,17,714,0,NaN,0,0,50,2132,False,22:00:00,2024-07-12,396,False
2024-07-12 23:00:00,US-SW-PNM,0,0,0,469,664,386,17,729,0,NaN,0,0,47,2267,False,23:00:00,2024-07-12,439,False


In [59]:
df_power_and_carbon.to_csv(f'data/{dates[0]}SWenergy_data.csv')